import pandas as pd
import plotly.express as px

data = pd.read_csv("C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\Part3) 파이썬 데이터 분석 실습\\Data\\실습5_ 교통 데이터를 활용한 지리 데이터 시각화\\서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv", encoding = 'cp949')


#- 날짜 컬럼 추가
data['연도'] = pd.to_datetime(data['사용월'], format='%Y%m').dt.year
data['월'] = pd.to_datetime(data['사용월'], format='%Y%m').dt.month

#- 2018년 이후, 2호선만 필터링
data = data.query('연도 >= 2018').query('호선명 == "2호선"')
data = data.query('호선명 == "2호선" and 연도 >= 2018')

#- 지하철역명 통일
data['지하철역'] = [i[0] for i in data['지하철역'].str.split('(')]

#- 승차 인원만 추출하기
on_col = [i for i in data.columns if '승차' in i]
data = data[['사용월', '연도','월','지하철역']+on_col]

## **분석**
### [1] 승차 인원이 가장 많은 역은?
#- 합계 컬럼 만들기
data['합계'] = data[on_col].sum(axis=1)
'''print(data.head())'''

#- 지하철역별 월평균 승차 인원 구하기
data_mean = data.groupby('지하철역')[['합계']].mean().reset_index().rename({'합계' : '월평균'}, axis=1).sort_values('월평균', ascending=False)

fig = px.bar(data_frame=data_mean, x='지하철역', y='월평균', title='지하철역별 월평균 승차인원')
'''fig.show()'''

### [2] 연도별로 혹은 월별로 승차 인원 추이에 차이가 있는가?
# - 연도별
year_sum = data.query('연도 <=2022').groupby(['연도'])[['합계']].sum().reset_index()
year_sum['연도'] = year_sum['연도'].astype(str)
fig = px.line(data_frame=year_sum, x='연도', y='합계')
'''fig.show()'''

# - 월별
month_sum = data.query('연도 <= 2022').groupby(['월'])[['합계']].sum().reset_index()
month_sum['월'] = month_sum['월'].astype(str)
fig = px.line(data_frame=month_sum, x='월', y='합계')
'''fig.show()'''

### [3] 시간대별로 가장 승차인원이 많은 역은?

#주요 역 10개만 필터링하여 시간당 월평균 인원수 구하기
top10 = data_mean.sort_values('월평균', ascending=False).head(10)['지하철역']
top10_mean_hour = data.query('지하철역 in @top10').groupby('지하철역')[on_col].mean()
top10_mean_hour.columns = [i[:3] for i in top10_mean_hour.columns]

#전체 기준 히트맵
top10_mean_hour.style.background_gradient(cmap='pink_r', axis=None).format('{:.0f}')

#행 기준 히트맵
top10_mean_hour.style.background_gradient(cmap='pink_r', axis=1).format('{:.0f}')


### [4] 지하철역 시간대별 인원 유형 군집화

#- 승차 인원으로 유형 군집화
hour_mean = data.groupby('지하철역')[on_col].mean()
hour_mean.columns = [i[:3] for i in hour_mean.columns]
# 행별 합이 1이 되는 퍼센트를 만든다
hour_mean_pct = hour_mean.div(hour_mean.sum(axis=1), axis=0)

from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,10))
visualizer.fit(hour_mean_pct)

k=3
model = KMeans(n_clusters = k, random_state = 10) # KMeans 모델을 정의
model.fit(hour_mean_pct) # fit을 통해 학습
hour_mean_pct['cluster'] = model.fit_predict(hour_mean_pct).astype(str) # 클러스터 열 만들기

fig = px.scatter(data_frame=hour_mean_pct[['08시','18시','cluster']].reset_index(), x='08시', y='18시', color='cluster', width=700, height=600, title='시간대별 승차 인원 비중 군집화', hover_name='지하철역')
'''fig.show()'''

'''for i in range(k):
    print(f'cluster {i}')
    print(list(hour_mean_pct.query(f'cluster == "{i}"').index)) '''

### [5] 지도에 분석 결과를 시각화

#-  8시 승차 인원과 18시 승차 인원을 지도에 시각화 해보자

#- 지하철역별 위도 경도 데이터 전처리

coordinate = pd.read_csv("C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\\Part3) 파이썬 데이터 분석 실습\\Data\\실습5_ 교통 데이터를 활용한 지리 데이터 시각화\\서울시 역사마스터 정보.csv", encoding='cp949')
'''print(coordinate.head())'''

coordinate = coordinate.query('호선 == "2호선"')
coordinate['역사명'] = [i[0] for i in coordinate['역사명'].str.split('(')]
coordinate.rename({'역사명' : '지하철역'}, axis=1, inplace=True)

hour_mean_merge = hour_mean.reset_index()[['지하철역','08시','18시']]
coordinate_merge = coordinate[['지하철역','위도','경도']]
hour_mean_coor = pd.merge(hour_mean_merge, coordinate_merge, on='지하철역')
hour_mean_coor['cluster'] = model.fit_predict(hour_mean_pct).astype(str)
'''print(hour_mean_coor.head())'''

#- 지도에 시각화
import folium
from folium import plugins

center = [37.541, 126.986]

m = folium.Map(location=center, zoom_start=12)
m.add_child(plugins.HeatMap(zip(hour_mean_coor['위도'],hour_mean_coor['경도'], hour_mean_coor['08시'])))


In [2]:
import pandas as pd
import plotly.express as px

data = pd.read_csv("C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\Part3) 파이썬 데이터 분석 실습\\Data\\실습5_ 교통 데이터를 활용한 지리 데이터 시각화\\서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv", encoding = 'cp949')

print(data)

<>:4: SyntaxWarning: invalid escape sequence '\P'
<>:4: SyntaxWarning: invalid escape sequence '\P'
C:\Users\User\AppData\Local\Temp\ipykernel_27116\240647074.py:4: SyntaxWarning: invalid escape sequence '\P'
  data = pd.read_csv("C:\\Users\\User\\Desktop\\패스트 캠퍼스 데이터 마스터\\파이썬\Part3) 파이썬 데이터 분석 실습\\Data\\실습5_ 교통 데이터를 활용한 지리 데이터 시각화\\서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv", encoding = 'cp949')


          사용월  호선명 지하철역  04시-05시 승차인원  04시-05시 하차인원  05시-06시 승차인원  \
0      202307  1호선  동대문           710            31         11581   
1      202307  1호선  동묘앞           221             2          3012   
2      202307  1호선  서울역           577            37          8434   
3      202307  1호선   시청            49             5          2381   
4      202307  1호선  신설동           392            35          8967   
...       ...  ...  ...           ...           ...           ...   
60633  201501  중앙선  운길산             0             0           177   
60634  201501  중앙선   원덕             1             0           292   
60635  201501  중앙선   중랑            14             1          4929   
60636  201501  중앙선   팔당             0             0           161   
60637  201501  중앙선   회기           436             3         10929   

       05시-06시 하차인원  06시-07시 승차인원  06시-07시 하차인원  07시-08시 승차인원  ...  \
0              2095          9027          6829         14177  ...   
1              1090          36